## Часть 1. [2 балла] Эксплоративный анализ
1. Найдите топ 10 (по частоте) именованных сущностей каждого из 6 типов.
2. Найдите топ 10 (по частоте) частотных триггеров событий.
3. Кластеризуйте все уникальные триггеры событий, используя эмбеддинги слов и любой алгоритм кластеризации (например, агломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий?

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=5461d67b879354b7c022594762a8a20705424a3e5dc81819043142c8c40dc07a
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!pip install GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
from tqdm import tqdm
import pandas as pd
from collections import defaultdict, Counter
from string import punctuation
import csv
import random
from sklearn.cluster import AgglomerativeClustering
import fasttext
import numpy as np
from git import Repo

In [ ]:
# Клонируем репозиторий
repo_url = 'https://github.com/dbamman/litbank.git'
repo_path = 'litbank'
if not os.path.exists(repo_path):
    Repo.clone_from(repo_url, repo_path)

In [ ]:
# Функция для разделения данных на обучающую, валидационную и тестовую выборки
def train_dev_test_split(data, train_percent=80, dev_percent=10, test_percent=10):
    random.shuffle(data)
    train_size = int(len(data) * train_percent / 100)
    train_data = data[:train_size]
    dev_size = int(len(data) * dev_percent / 100)
    dev_data = data[train_size:train_size + dev_size]
    test_size = int(len(data) * test_percent / 100)
    test_data = data[train_size + dev_size:]
    return train_data, dev_data, test_data

In [ ]:
# Функция для подготовки предложений и меток из файлов
def prepare_sents(root, files):
    sents, labels = [], []
    for f in tqdm(files):
        path = os.path.join(root, f)
        df = pd.read_csv(path, sep='\t', quoting=csv.QUOTE_NONE, header=None)
        words = list(df[0])
        labels = list(df[1])

        sentences = [[]]
        labels_sent = [[]]

        for i in range(len(words)):
            if words[i] not in ['.', '!', '?', '...', '', ',']:
                if words[i] in punctuation:
                    continue
                sentences[-1].append(words[i].lower())
                labels_sent[-1].append(labels[i])
            elif sentences[-1] != []:
                sentences.append([])
                labels_sent.append([])

        if sentences[-1] == []:
            sentences = sentences[:-1]
            labels_sent = labels_sent[:-1]
        sents += sentences
        labels += labels_sent

    return sents, labels

In [ ]:
# entities
root_entities = '/content/litbank/entities/brat'
files_entities = []

for dirpath, dirnames, filenames in os.walk(root_entities):
  for filename in filenames:
    if filename.endswith('ann'):
      files_entities.append(filename)

files_entities = sorted(files_entities)

entities = defaultdict(Counter)
for file in tqdm(files_entities):
    data = pd.read_csv(os.path.join(root_entities, file), sep='\t', header = None, quoting=3)
    data[1] = data[1].apply(lambda x: x.split(' ')[0])
    for i, row in data.iterrows():
        entities[row[1]][row[2]] += 1

In [ ]:
data[2] = data[2].astype(str)
grouped_data = data.groupby(1)[[1, 2]].apply(lambda x: Counter(tuple(i) for i in x.values)).to_dict()

# Топ 10 entities из каждой категории
for entity_type, entity_counts in grouped_data.items():
    print(entity_type)
    for (entity, text), count in entity_counts.most_common(10):
        print(f"('{text}', {count})")
    print()

FAC
('his father 's farm', 1)
('Wall Street', 1)
('home', 1)
('his bedroom', 1)
('the music room of their house on Washington Square', 1)
('their house on Washington Square', 1)
('Washington Square', 1)
('grampa 's in Tarrytown', 1)
('the best hotel in Lucerne', 1)
('her own drawing room', 1)

GPE
('Tarrytown', 3)
('New York', 2)
('Boston', 2)
('America', 2)
('Rome', 2)
('Atlantic City', 1)
('England', 1)
('Switzerland', 1)
('Philadelphia', 1)
('Lucerne', 1)

LOC
('a clean pond', 1)
('the sea', 1)
('the world', 1)

ORG
('Harvard', 3)
('New York cavalry regiment', 1)
('clubs', 1)
('another choir', 1)
('a half dozen " Stamp and Coin " companies', 1)
('college', 1)

PER
('Anthony', 10)
('his father', 3)
('his grandfather', 3)
('Anthony Patch', 2)
('Adam J. Patch', 2)
('his grandson', 2)
('son', 2)
('intelligent men', 1)
('a man', 1)
('the crusaders', 1)

VEH
('tandems', 1)



In [ ]:
# Загрузка данных для кластеризации триггеров событий
root_events = '/content/litbank/events/brat'
files_events = []

for dirpath, dirnames, filenames in os.walk(root_events):
    for filename in filenames:
        if filename.endswith('ann'):
            files_events.append(filename)

files_events = sorted(files_events)
events = Counter()

for file in tqdm(files_events):
    try:
        data = pd.read_csv(os.path.join(root_events, file), sep='\t', header=None, quoting=3)
    except pd.errors.EmptyDataError:
        continue
    data[1] = data[1].apply(lambda x: x.split(' ')[0])
    for i, row in data.iterrows():
        events[row[2]] += 1

# Найдите топ 10 (по частоте) частотных триггеров событий.
print(*events.most_common(10), sep='\n')

100%|██████████| 100/100 [00:01<00:00, 94.20it/s]


('said', 464)
('came', 95)
('looked', 92)
('went', 92)
('asked', 69)
('heard', 63)
('saw', 59)
('cried', 59)
('took', 56)
('turned', 55)


In [ ]:
# Кластеризация триггеров событий
root_tsv = '/content/litbank/events/tsv'
files_tsv = sorted(list(os.walk(root_tsv)))[0][-1]

sentences, _ = prepare_sents(root_tsv, files_tsv)

# Разделение данных на обучающую, валидационную и тестовую выборки
train_data, dev_data, test_data = train_dev_test_split(sentences, 80, 10, 10)

100%|██████████| 100/100 [00:00<00:00, 166.67it/s]


In [ ]:
# Запись данных в файл для обучения fastText
with open('train_text.txt', 'w') as f:
    for s in sentences:
        f.write(' '.join(s))
        f.write('\n')

# Обучение fastText
ft = fasttext.train_unsupervised('train_text.txt')

# Получение эмбеддингов триггеров
trigger_embeddings = []
triggers = []

for trigger in events.keys():
    triggers.append(trigger)
    trigger_embeddings.append(ft.get_word_vector(trigger.lower()))

trigger_embeddings = np.array(trigger_embeddings)

In [ ]:
# Кластеризация с использованием агломеративного алгоритма
clustering = AgglomerativeClustering(n_clusters=3)
clustering.fit(trigger_embeddings)

# Печать результатов кластеризации
clusters = defaultdict(list)
for word, cluster in zip(triggers, clustering.labels_):
    clusters[cluster].append(word)

for cluster, words in clusters.items():
    print(f"Cluster {cluster}: {words}")

Cluster 0: ['Smoke', 'losing', 'sliding', 'sticking', 'Fog', 'fog', 'flows', 'rolls', 'creeping', 'hovering', 'drooping', 'pinching', 'peeping', 'Gas', 'lighted', 'gas', 'addressed', 'contemplation', 'engaged', 'groping', 'running', 'mentioned', 'come', 'plants', 'drones', 'blew', 'observed', 'handled', 'correcting', 'acquired', 'infection', 'see', 'pretence', 'lowering', 'temper', 'born', 'married', 'died', 'issue', 'adding', 'Married', 'birth', 'inserting', 'lost', 'settled', 'death', 'introduction', 'forced', 'found', 'invited', 'seen', 'expected', 'tidings', 'uniting', 'retired', 'entered', 'brought', 'welded', 'resolved', 'provided', 'entertained', 'held', 'appearances', 'fire', 'streamed', 'look', 'revel', 'dreams', 'glitter', 'fancies', 'stalked', 'writhed', 'taking', 'causing', 'strikes', 'voice', 'chime', 'die', 'endured', 'laughter', 'depart', 'swells', 'stream', 'waning', 'peal', 'indulged', 'reaches', 'beat', 'sounding', 'ceased', 'music', 'quieted', 'cessation', 'revelled'

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
embed_tsne = tsne.fit_transform(trigger_embeddings)

In [ ]:
fig = px.scatter(x=embed_tsne[:,0], y=embed_tsne[:,1], text=triggers, color=clustering.labels_)
fig.update_traces(textfont_size=2)
# fig.update_annotations(textfont_size=2) - для того, чтобы разобрать слова на графике
fig.show()

## Часть 2. [3 балла] Извлечение именованных сущностей
1. Обучите стандартную модель для извлечения именованных сущностей, CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей.
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация).
2. Замените часть модели на символах и словах (CNN + эмбеддинги словах) на ELMo и / или BERT. Должна получиться модель ELMo / BERT + BiLSTM + CRF.
3. Замените модель последовательности (BiLSTM) на другой слой, например, на Transformer. Должна получиться модель CNN  + Transformer + CRF.

[бонус] Дообучите BERT для извлечения именованных сущностей.

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]

[бонус] Модифицируйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]: вместо эмбеддингов слов используйте ELMo и/или BERT.

In [ ]:
import warnings
from typing import List
from typing import Union, Tuple, Dict
import torch
from sklearn.metrics import f1_score, classification_report
import random
from string import punctuation

In [ ]:
data_directory = '/content/litbank/entities/tsv'
file_list = sorted(list(os.walk(data_directory))[0][-1])

warnings.filterwarnings("ignore")

In [ ]:
def split_train_dev_test(data, train_percent=80, dev_percent=10, test_percent=10):
    random.shuffle(data)
    train_size = int(len(data) * train_percent / 100)
    train_data = data[:train_size]
    dev_size = int(len(data) * dev_percent / 100)
    dev_data = data[train_size:train_size + dev_size]
    test_size = int(len(data) * test_percent / 100)
    test_data = data[train_size + dev_size:]
    return train_data, dev_data, test_data

In [ ]:
def prepare_sentences(directory, file_list, verbose=True):
    sentences, labels = [], []
    for file_name in tqdm(file_list, disable=not verbose):
        path = os.path.join(directory, file_name)
        with open(path) as file:
            content = file.readlines()
        sentence_temp = []
        labels_temp = []
        for line in content:
            if line == '\n':
                if sentence_temp:
                    sentences.append(sentence_temp)
                    labels.append(labels_temp)
                sentence_temp = []
                labels_temp = []
            else:
                line_parts = line.split('\t')
                sentence_temp.append(line_parts[0])
                labels_temp.append(line_parts[1])
        if sentence_temp:
            sentences.append(sentence_temp)
            labels.append(labels_temp)
    return sentences, labels


In [ ]:
root_directory = '/content/litbank/entities/tsv'
files_list = sorted(list(os.walk(root_directory))[0][-1])

In [ ]:
# Разделим как рекомендовано на train, test и dev
train_data, dev_data, test_data = split_train_dev_test(files_list, 80, 10, 10)

train_books = [os.path.join(root_directory, file) for file in train_data]
dev_books = [os.path.join(root_directory, file) for file in dev_data]
test_books = [os.path.join(root_directory, file) for file in test_data]

train_sentences, train_labels = prepare_sentences(root_directory, train_books)
dev_sentences, dev_labels = prepare_sentences(root_directory, dev_books)
test_sentences, test_labels = prepare_sentences(root_directory, test_books)

100%|██████████| 10/10 [00:00<00:00, 362.30it/s]


In [ ]:
df_train = pd.DataFrame(zip(train_sentences, train_labels), columns=['sentence', 'labels'])
df_dev = pd.DataFrame(zip(dev_sentences, dev_labels), columns=['sentence', 'labels'])
df_test = pd.DataFrame(zip(test_sentences, test_labels), columns=['sentence', 'labels'])

df_train['chars'] = df_train['sentence'].apply(lambda x: [list(y) for y in x])
df_dev['chars'] = df_dev['sentence'].apply(lambda x: [list(y) for y in x])
df_test['chars'] = df_test['sentence'].apply(lambda x: [list(y) for y in x])

In [ ]:
def build_token_dict(tokens: List, special_tokens: List) -> Tuple[Dict]:
    token2idx = defaultdict(lambda: 0)
    idx2token = []

    for index, stoken in enumerate(special_tokens):
        token2idx[stoken] = index

    unique_tokens = set()
    for item in tokens:
        for token in item:
            if token not in special_tokens:
                unique_tokens.add(token)

    for index, utoken in enumerate(unique_tokens, len(special_tokens)):
        token2idx[utoken] = index

    sorted_dict = sorted(token2idx.items(), key=lambda x: x[1])

    for elem in sorted_dict:
        idx2token.append(elem[0])

    return token2idx, idx2token

In [ ]:
def create_embedding_matrix(word_vecs, token2idx: Dict, emb_size: int = 300, special_ids: Tuple[int] = (0, 1)) -> np.ndarray:
    emb_matrix = np.zeros((len(token2idx), emb_size), dtype="float32")

    emb_matrix[special_ids[0]] = np.zeros(emb_size, dtype='float32')
    emb_matrix[special_ids[1]] = np.random.uniform(-0.25, 0.25, emb_size)

    for token, id in token2idx.items():
        if id not in special_ids:
            if token in word_vecs:
                emb_matrix[id] = word_vecs[token]
            else:
                emb_matrix[id] = np.random.uniform(-0.25, 0.25, emb_size)
    return emb_matrix


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def label_prediction(model, iterator, TAG_PAD_IDX=0, exclude_pad=True, device='cpu'):

    model.eval()

    tokens, labels, pred_labels = [], [], []
    with torch.no_grad():
        for batch in iterator:
            predictions, _ = model(batch['tokens'].to(device), batch['chars'].to(device), batch['mask'].to(device), batch['labels'].to(device))

            for sent_texts, sent_tags_true, preds in zip(batch['tokens'], batch['labels'], predictions):
                sent_tags_true = sent_tags_true.cpu().numpy()
                sent_texts = sent_texts.cpu().numpy()
                preds = preds.cpu().numpy()
                if exclude_pad:
                    args = np.where(sent_texts != TAG_PAD_IDX)[0]

                    tokens.append(list(sent_texts[args]))
                    labels.append(list(sent_tags_true[args]))
                    pred_labels.append(list(preds[args]))

    return tokens, labels, pred_labels

In [ ]:
def train_model(model, train_dataloader, valid_dataloader, optimizer, idx2tag, epochs=5, pad_idx=0, clip=1):
    best_acc = 0
    best_loss = 1000

    for epoch_num in range(epochs):
        classes = []
        predicted_classes = []
        epoch_acc_train = 0
        epoch_loss_train = 0

        model.train()

        for batch in tqdm(train_dataloader):
            optimizer.zero_grad()

            try:
                predictions, loss = model(batch['tokens'].to(device), batch['chars'].to(device), batch['mask'].to(device), batch['labels'].to(device))
            except Exception as e:
                print(f"Error in batch: {e}")
                print(f"Batch: {batch}")
                raise e

            train_label = batch['labels'].to(device).view(-1).cpu()
            predictions = predictions.view(-1).cpu()
            texts = batch['tokens'].to(device).view(-1).cpu()

            true_pred = ((train_label == predictions) & (texts != pad_idx)).sum()
            num_pred = (texts != pad_idx).sum()
            acc = true_pred / num_pred
            epoch_acc_train += acc

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
            optimizer.step()
            epoch_loss_train += loss.item()

        train_accuracy = epoch_acc_train / len(train_dataloader)
        train_loss = epoch_loss_train / len(train_dataloader)

        model.eval()

        epoch_acc_valid = 0
        epoch_loss_valid = 0

        for batch in valid_dataloader:
            optimizer.zero_grad()

            try:
                predictions, loss = model(batch['tokens'].to(device), batch['chars'].to(device), batch['mask'].to(device), batch['labels'].to(device))
            except Exception as e:
                print(f"Error in batch: {e}")
                print(f"Batch: {batch}")
                raise e

            epoch_loss_valid += loss.item()

            valid_label = batch['labels'].to(device).view(-1).cpu()
            predictions = predictions.view(-1).cpu()
            texts = batch['tokens'].to(device).view(-1).cpu()

            true_pred = ((valid_label == predictions) & (texts != pad_idx)).sum()
            num_pred = (texts != pad_idx).sum()
            acc = true_pred / num_pred
            epoch_acc_valid += acc

            predicted_classes.extend(predictions.cpu().numpy())
            classes.extend(valid_label.cpu().numpy())

        valid_accuracy = epoch_acc_valid / len(valid_dataloader)
        valid_loss = epoch_loss_valid / len(valid_dataloader)

        print(f'Epochs: {epoch_num + 1} | Accuracy: {valid_accuracy: .3f}')


In [ ]:
def create_custom_dataset(df, token2id, label2id, char2id, text_col='context_processed',
                          label_col='spell', char_col='characters'):
    pad_to=124
    pad_to_char=30
    pad_value=0
    unk_value=1
    pad_left=False
    pad_left_char=False
    is_char=True

    tokens = df[text_col].tolist()
    chars = df[char_col].tolist()
    labels = df[label_col].tolist()

    dataset = []

    for i in range(len(tokens)):
        if is_char:
            token, char, tag = tokens[i], chars[i], labels[i]
            token, char, tag = transform_data(token, token2id, pad_to, pad_value, unk_value, pad_left, kind='token'), \
                               transform_data(char, char2id, pad_to_char, pad_value, unk_value, pad_left_char,
                                              kind='char', is_char=True), \
                               transform_data(tag, label2id, pad_to, pad_value, unk_value, pad_left, kind='tag')
            mask = [0 if t == pad_value else 1 for t in token]
            mask = torch.tensor(mask, dtype=torch.bool)
            entry = {'tokens': token, 'chars': char, 'labels': tag, 'mask': mask}
        else:
            token, tag = tokens[i], labels[i]
            token, tag = transform_data(token, token2id, pad_to, pad_value, unk_value, pad_left), \
                         transform_data(tag, label2id, pad_to, pad_value, unk_value, pad_left)
            entry = {'tokens': token, 'labels': tag}

        dataset.append(entry)

    return dataset


def transform_data(data, mapping, pad_to, pad_value, unk_value, pad_left, kind='token', is_char=False):
    pad_to_char=30
    pad_left_char=False
    if kind == 'token':
        if len(data) != pad_to:
            if len(data) > pad_to:
                data = [mapping[i] if i in mapping else unk_value for i in data[:pad_to]]
            else:
                n_pads = pad_to - len(data)
                if kind == 'token':
                    data = [pad_value] * n_pads * pad_left + [mapping[i] if i in mapping else unk_value for i in
                                                               data] + [pad_value] * n_pads * (not pad_left)
                else:
                    data = [mapping['O']] * n_pads * pad_left + [mapping[i] if i in mapping else unk_value for i in
                                                                data] + [mapping['O']] * n_pads * (not pad_left)

        return torch.tensor(data, dtype=torch.int)

    elif is_char:
        chars_embeddings = []
        for word in data:
            if len(word) != pad_to:
                if len(word) > pad_to:
                    word = [mapping[i] if i in mapping else unk_value for i in word[:pad_to]]
                else:
                    n_pads = pad_to - len(word)
                    word = [pad_value] * n_pads * pad_left + [mapping[i] if i in mapping else unk_value for i in
                                                               word] + [pad_value] * n_pads * (
                                       not pad_left)
            else:
                word = [mapping[i] if i in mapping else unk_value for i in word]
            chars_embeddings.append(word)

        if pad_to is not None and len(chars_embeddings) != pad_to:
            if len(chars_embeddings) > pad_to:
                chars_embeddings = chars_embeddings[:pad_to]
            else:
                n_pads = pad_to - len(chars_embeddings)
                chars_embeddings = [[pad_value] * pad_to_char] * n_pads * pad_left_char + chars_embeddings + [
                    [pad_value] * pad_to_char] * n_pads * (not pad_left_char)

        return torch.tensor(chars_embeddings, dtype=torch.int)


In [ ]:
import gensim

In [ ]:
! wget https://www.dropbox.com/s/699kgut7hdb5tg9/GoogleNews-vectors-negative300.bin.gz?dl=1
! mv 'GoogleNews-vectors-negative300.bin.gz?dl=1' GoogleNews-vectors-negative300.bin.gz
! gunzip GoogleNews-vectors-negative300.bin.gz

--2023-11-20 21:07:43--  https://www.dropbox.com/s/699kgut7hdb5tg9/GoogleNews-vectors-negative300.bin.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/699kgut7hdb5tg9/GoogleNews-vectors-negative300.bin.gz [following]
--2023-11-20 21:07:43--  https://www.dropbox.com/s/dl/699kgut7hdb5tg9/GoogleNews-vectors-negative300.bin.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc9d781cb8721d56f57298b30aa.dl.dropboxusercontent.com/cd/0/get/CH7rdFR7Emkxud5PSBdtvFnw3ABaULhNKUikPfRQSvBMzIKOszr81JWGJazsE_XNuvvorOn1A25sYw4R4B28gwu49mD7Kn7p-_C_qloi_pO72cZ8p8qYIsw6Ifgvo_phfMg/file?dl=1# [following]
--2023-11-20 21:07:43--  https://ucc9d781cb8721d56f57298b30aa.dl.dropboxusercontent.com/cd/0/get/CH7rdFR7Emkxud5PSBdtvFnw3ABaULhNKUikPfRQSvBMzI

In [ ]:
# Определение функции load_word_vectors
def load_word_vectors(file_path: str) -> Dict:
    word_vecs = {}
    with gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=True) as word_model:
        for word in word_model.vocab:
            word_vecs[word] = word_model[word]
    return word_vecs

In [ ]:
# Построение словарей для токенов, меток и символов
token2idx, idx2token = build_token_dict(df_train['sentence'].tolist() + df_dev['sentence'].tolist(), ['<PAD>', '<UNK>'])
tag2idx, idx2tag = build_token_dict(df_train["labels"], [])
char2idx, idx2char = build_token_dict(df_train['sentence'].tolist() + df_dev['sentence'].tolist(), ['<PAD>', '<UNK>'])

In [ ]:
# Создание наборов данных
dataset_train = create_custom_dataset(df_train, token2idx, tag2idx, char2idx, text_col='sentence', label_col='labels', char_col='chars')
dataset_dev = create_custom_dataset(df_dev, token2idx, tag2idx, char2idx, text_col='sentence', label_col='labels', char_col='chars')
dataset_test = create_custom_dataset(df_test, token2idx, tag2idx, char2idx, text_col='sentence', label_col='labels', char_col='chars')

In [ ]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
word2vec_embeddings = create_embedding_matrix(word2vec, token2idx, emb_size=300)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
# Определение загрузчиков данных
BATCH_SIZE = 32
train_dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dev_dataloader = DataLoader(dataset_dev, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
! pip install pytorch-crf

In [ ]:
from torchcrf import CRF
from torch import nn
from torch.nn import functional as F
#from sklearn.metrics import classification_report

In [ ]:
# На данный момент решения, это просто какая-то адски сложная часть, которая постоянно вызывает кучу ошибок, пришлось подсмотреть идею реализации уодной из групп и это просто какая-то ФАНТАСТИКА, что оно в итоге заработало

class cnn_biLSTM_crf(nn.Module):
    def __init__(self, word2id, char2id, idx2tag, num_classes, device,
                 word_embedding_dim=300, char_embedding_dim=20, num_filters=100,
                 hidden_dim=200, num_layers=2, filter_size=3, drop_out=0.5,
                 pad_idx=0, pretrained_embedding=None, freeze_embedding=False):

        super(cnn_biLSTM_crf, self).__init__()

        self.device = device
        self.pad_idx = pad_idx
        self.idx2tag = idx2tag

        self.word_embedding = nn.Embedding(len(word2id), word_embedding_dim, padding_idx=pad_idx)

        if type(pretrained_embedding) == np.ndarray:
            self.word_embedding.weight.data.copy_(torch.from_numpy(pretrained_embedding))
            self.word_embedding.weight.requires_grad = True

        self.embedding_dim = char_embedding_dim
        self.char_embedding = nn.Embedding(len(char2id), char_embedding_dim)
        self.char_embedding.weight.requires_grad = True
        self.cnn = nn.Conv3d(in_channels=1, out_channels=num_filters, kernel_size=(1, filter_size, char_embedding_dim))

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn_input_dim = word_embedding_dim + num_filters
        self.lstm = nn.LSTM(self.rnn_input_dim, hidden_dim // 2, num_layers,
                            bidirectional=True, batch_first=True, dropout=drop_out)
        self.fc = nn.Linear(hidden_dim, num_classes)

        self.crf = CRF(num_classes, batch_first=True)

    def forward(self, words_to_id, chars_to_id, mask, label=None):
        word_embedding = self.word_embedding(words_to_id.to(self.device))

        max_len, max_len_char = chars_to_id.size(1), chars_to_id.size(2)
        inputs = chars_to_id.view(-1, max_len * max_len_char)
        input_embed = self.char_embedding(inputs)
        input_embed = input_embed.view(-1, 1, max_len, max_len_char, self.embedding_dim)
        conv_output = self.cnn(input_embed)
        pool_output = torch.squeeze(torch.max(conv_output, -2)[0])
        char_embedding = pool_output.transpose(-2, -1).contiguous()

        embedding = torch.cat([word_embedding, char_embedding], 2)

        self.lstm.flatten_parameters()
        out, (h, c) = self.lstm(embedding)
        out = self.fc(out)

        if label is not None:
            loss = -self.crf(out, label, mask, reduction='mean')
        else:
            loss = None

        predicted_id = self.crf.decode(out, mask)
        seq_len = label.shape[-1]
        for i, j in enumerate(predicted_id):
            predicted_id[i] += [0] * (seq_len - mask[i, :].sum())
        predicted_id = torch.tensor(predicted_id, dtype=torch.int)

        return predicted_id, loss

    def compute_all(self, batch):
        words_to_id = batch['tokens']
        chars_to_id = batch['chars']
        label = batch['labels']
        mask = batch['mask']
        out = self.forward(words_to_id, chars_to_id)
        loss = -self.crf(out, label, mask, reduction='mean')

        seq_len = label.shape[-1]
        predicted_id = self.crf.decode(out, mask)

        for i, j in enumerate(predicted_id):
            predicted_id[i] += [0] * (seq_len - mask[i, :].sum())
        predicted_id = torch.tensor(predicted_id, dtype=torch.int).to(self.device)

        true_pred = ((label == predicted_id) & (label != self.pad_idx)).sum()
        num_pred = (label != self.pad_idx).sum()
        acc = (true_pred / num_pred).cpu()
        metrics = dict(acc=acc)

        return loss, metrics

    def decode(self, words_to_id, chars_to_id, mask):
        out = self.forward(words_to_id, chars_to_id)
        predicted_id = self.crf.decode(out, mask)
        return predicted_id

    def _eval(self, valid_dataloader):
        self.eval()

        true_labels = []
        predict_labels = []

        for batch in valid_dataloader:
            batch = {k: v.to(self.device) for k, v in batch.items()}
            words_to_id = batch['tokens'].to(self.device)
            chars_to_id = batch['chars'].to(self.device)
            label = batch['labels'].to(self.device)
            mask = batch['mask'].to(self.device)

            out = self.decode(words_to_id, chars_to_id, mask)

            for out_sentence, label_sentence in zip(out, label.tolist()):
                for predict_label, true_label in zip(out_sentence, label_sentence):
                    true_labels.append(self.idx2tag[true_label])
                    predict_labels.append(self.idx2tag[predict_label])
        report = classification_report(true_labels, predict_labels, output_dict=True)
        return report


In [ ]:
N_CLASSES = len(tag2idx)
model = cnn_biLSTM_crf(token2idx,
                                  char2idx,
                                  idx2tag,
                                  N_CLASSES,
                                  device=device,
                                  hidden_dim=512,
                                  num_layers=2,
                                  drop_out=0.2,
                                  pretrained_embedding=word2vec_embeddings
                                  )
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

model.apply(init_weights)
model.to(device)

cnn_biLSTM_crf(
  (word_embedding): Embedding(16761, 300, padding_idx=0)
  (char_embedding): Embedding(16761, 20)
  (cnn): Conv3d(1, 100, kernel_size=(1, 3, 20), stride=(1, 1, 1))
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=13, bias=True)
  (crf): CRF(num_tags=13)
)

In [ ]:
import torch.optim as optim

In [ ]:
# Определение оптимизатора
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model, train_dataloader, dev_dataloader, optimizer, idx2tag, epochs=10, pad_idx=0)

100%|██████████| 218/218 [12:12<00:00,  3.36s/it]


Epochs: 1 | Accuracy:  0.899


In [ ]:
# Оценка на тестовом наборе
tokens, labels, pred_labels = label_prediction(model, test_dataloader, TAG_PAD_IDX=0, exclude_pad=True)
flat_labels = [el for subset in labels for el in subset]
flat_preds = [el for subset in pred_labels for el in subset]

true_labels = [idx2tag[l] for l in flat_labels]
label_prediction = [idx2tag[l] for l in flat_preds]
labels = [i for i in idx2tag if i not in ('<PAD>', '<UNK>')]

In [ ]:
# Вывод отчета по классификации
print(classification_report(true_labels, label_prediction, labels=labels))

              precision    recall  f1-score   support

           O       0.91      0.97      0.94     17577
       B-VEH       0.00      0.00      0.00        21
       I-FAC       0.60      0.25      0.36       362
       B-FAC       0.48      0.34      0.40       221
       I-VEH       0.00      0.00      0.00        25
       B-ORG       0.00      0.00      0.00        15
       I-PER       0.60      0.39      0.47      1189
       I-ORG       0.00      0.00      0.00        28
       I-GPE       0.75      0.06      0.11        53
       B-LOC       0.92      0.13      0.22        94
       B-PER       0.62      0.57      0.59       866
       B-GPE       0.80      0.05      0.10        79
       I-LOC       0.82      0.12      0.20       120

    accuracy                           0.89     20650
   macro avg       0.50      0.22      0.26     20650
weighted avg       0.87      0.89      0.87     20650



## Часть 3. [2 балла] Извлечение событий

1. Используйте BiLSTM на эмбеддингах слов для извлечения триггеров событий.

2. Замените часть модели на  словах  на ELMo и/или BERT.  Должна получиться модель ELMo / BERT + BiLSTM.

[бонус] Предобучите BiLSTM как языковую модель. Дообучите ее для извлечения триггеров.

[бонус] Дообучите BERT для извлечения триггеров событий.

In [ ]:
!git clone https://github.com/dbamman/litbank.git

Cloning into 'litbank'...
remote: Enumerating objects: 1187, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1187 (delta 26), reused 22 (delta 22), pack-reused 1149
Receiving objects: 100% (1187/1187), 40.65 MiB | 14.76 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [ ]:
!pip install transformers

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from collections import defaultdict, Counter

import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import f1_score

from yellowbrick.cluster import KElbowVisualizer

import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

from transformers import AutoTokenizer, AutoModel

import json
import shutil
from copy import deepcopy
import random
from string import punctuation

In [ ]:
root = '/content/litbank/events/tsv'
files = sorted(list(os.walk(root))[0][-1])

In [ ]:
def train_dev_test_split(data, train_percent = 80, dev_percent = 10, test_percent = 10):
  random.shuffle(data)
  train_size = int(len(data) * train_percent / 100)
  train_data = data[:train_size]
  dev_size = int(len(data) * dev_percent / 100)
  dev_data = data[train_size:train_size+dev_size]
  test_size = int(len(data) * test_percent / 100)
  test_data = data[train_size+dev_size:]
  return train_data, dev_data, test_data

In [ ]:
train_data, dev_data, test_data = train_dev_test_split(files, 80, 10, 10)

In [ ]:
def prepare_sents(root, files, verbose=True):
    sents, labels = [], []
    for f in tqdm(files, disable=not verbose):
        path = os.path.join(root, f)
        df = pd.read_csv(path, sep='\t', quoting=3, header=None)
        words = list(df[0])
        labels_list = list(df[1])
        sentences = []
        labels_sent = []
        for i in range(len(words)):
            if words[i] not in ['.', '!', '?', '...', '']:
                if words[i] in punctuation:
                    continue
                sentences.append(words[i].lower())
                labels_sent.append(labels_list[i])
            elif sentences != []:
                sents.append(sentences)
                labels.append(labels_sent)
                sentences = []
                labels_sent = []
    return sents, labels

sents_train, labels_train = prepare_sents(root, train_data)
sents_dev, labels_dev = prepare_sents(root, dev_data)
sents_test, labels_test = prepare_sents(root, test_data)

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

In [ ]:
def vectorizing(sent, embed_dim=768):
    sent_raw = ' '.join(sent)
    encoded = bert_tokenizer.encode_plus(sent_raw, return_tensors="pt")
    with torch.no_grad():
        output = bert_model(**encoded)
        states = output.hidden_states
        concat = torch.stack(states[-4:]).sum(0).squeeze()
    embeds = np.zeros((len(sent), embed_dim))
    for idx in range(len(sent)):
        token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
        embed = concat[token_ids_word].mean(dim=0)
        embeds[idx] = embed
    return embeds

In [ ]:
embeds_train = []
for sent in tqdm(sents_train):
    embed = vectorizing(sent)
    embeds_train.append(embed)

  0%|          | 0/7101 [00:00<?, ?it/s]

In [ ]:
embeds_dev = []
for sent in tqdm(sents_dev):
    embed = vectorizing(sent)
    embeds_dev.append(embed)

  0%|          | 0/777 [00:00<?, ?it/s]

In [ ]:
def prepare_sample(sent, label):
    sent_len = len(sent)
    label = (np.array(label) == 'EVENT').astype(np.int32)
    return {
        'sample': sent,
        'label': label,
        'length': sent_len
    }

def create_custom_dataset(sents, labels):
    dataset = []
    for idx in range(len(sents)):
        sample = prepare_sample(sents[idx], labels[idx])
        dataset.append(sample)
    return dataset

In [ ]:
train_ds = create_custom_dataset(embeds_train, labels_train)
val_ds = create_custom_dataset(embeds_dev, labels_dev)

In [ ]:
def init_weights(embedder, hidden2tag):
    nn.init.xavier_uniform_(embedder.weight)
    nn.init.xavier_uniform_(hidden2tag.weight)
    embedder.bias.data.zero_()
    hidden2tag.bias.data.zero_()

def bilstm_model(embedding_dim=768, hidden_dim=256, num_layers=1, lr_scheduler=None, lr_scheduler_type=None):
    rnn = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=num_layers, bidirectional=True, dropout=0.3)
    embedder = nn.Linear(embedding_dim, embedding_dim // 2)
    hidden2tag = nn.Linear(hidden_dim, 2)

    init_weights(embedder, hidden2tag)

    def forward(x, length):
        packed = pack_padded_sequence(x, length, batch_first=True, enforce_sorted=False)
        output, _ = rnn(packed)
        return hidden2tag(output.data)

    def compute_all(batch):
        sample = batch['sample'].float()
        length = batch['length'].cpu()
        labels = batch['label']

        logits = forward(sample, length)

        labels = pack_padded_sequence(labels.float(), length, batch_first=True, enforce_sorted=False).data

        criterion = nn.CrossEntropyLoss(weight=torch.tensor([1., 24.]))
        loss = criterion(logits, labels.long())
        acc = f1_score(labels.long().cpu().numpy(), torch.argmax(logits, axis=1).detach().cpu().numpy())

        metrics = dict(acc=acc, loss=loss.item())
        return loss, metrics

    def post_train_batch():
        if lr_scheduler is not None and lr_scheduler_type == 'per_batch':
            lr_scheduler.step()

    def post_val_stage(val_loss):
        if lr_scheduler is not None and lr_scheduler_type == 'per_epoch':
            lr_scheduler.step(val_loss)

    return forward, compute_all, post_train_batch, post_val_stage

In [ ]:
def collate_fn(batch):
    return {'sample': torch.nn.utils.rnn.pad_sequence([torch.tensor(d['sample']) for d in batch], batch_first=True),
            'label': torch.nn.utils.rnn.pad_sequence([torch.tensor(d['label']) for d in batch], batch_first=True),
            'length': torch.tensor([d['length'] for d in batch])}

In [ ]:
class model_training:
    def __init__(self, model, optimizer, train_loader, val_loader, output_folder: str = '/content/drive/MyDrive/rbg_unet/', batch_size: int = 4):
        self.output_folder = output_folder
        self.tboard_log_dir = './tboard_logs/'
        self.model = model
        self.optimizer = optimizer
        self.train_dataset = train_loader
        self.val_dataset = val_loader
        self.batch_size = batch_size

        shutil.rmtree(self.tboard_log_dir, ignore_errors=True)
        if os.path.exists(self.output_folder):
            self.load_checkpoint()
        else:
            os.makedirs(self.output_folder)
            self.global_step = 0
            self.prev_epoch = 0
            self.best_loss = float('inf')
            self.global_step = 0
        self.train_writer = SummaryWriter(log_dir=self.tboard_log_dir + "train/")
        self.val_writer = SummaryWriter(log_dir=self.tboard_log_dir + "val/")
        self.cache = self.cache_states()

    def load_checkpoint(self):
        checkpoint = torch.load(os.path.join(self.output_folder, 'last_checkpoint.pth'))
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.prev_epoch = checkpoint['epoch_num']
        self.best_loss = checkpoint['loss']
        self.global_step = checkpoint['global_step']
        shutil.copytree(os.path.join(self.output_folder, 'tboard_logs'), self.tboard_log_dir)
        return

    def save_checkpoint(self, path, model_state_dict, optimizer_state_dict, loss, epoch_num):
        if os.path.exists(os.path.join(self.output_folder, 'tboard_logs')):
            shutil.rmtree(os.path.join(self.output_folder, 'tboard_logs'))
        shutil.copytree(self.tboard_log_dir, os.path.join(self.output_folder, 'tboard_logs'))
        torch.save({
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'loss': loss,
            'epoch_num': epoch_num,
            'global_step': self.global_step,
        }, path)

    def train(self, num_epochs: int):
        model = self.model
        optimizer = self.optimizer

        for epoch in range(self.prev_epoch + 1, self.prev_epoch + num_epochs + 1):
            model.train()
            for batch in tqdm(train_loader, desc='Epoch {}'.format(epoch)):
                batch = {k: v.float() for k, v in batch.items()}
                loss, details = model.compute_all(batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                model.post_train_batch()
                for k, v in details.items():
                    self.train_writer.add_scalar(k, v, global_step=self.global_step)
                self.train_writer.flush()
                self.global_step += 1

            model.eval()
            val_accs = []
            val_losses = []
            val_logs = defaultdict(list)
            for batch in tqdm(val_loader):
                batch = {k: v.float() for k, v in batch.items()}
                loss, details = model.compute_all(batch)
                val_losses.append(loss.item())
                val_accs.append(details['acc'])
                for k, v in details.items():
                    val_logs[k].append(v)
            val_logs = {k: np.mean(v) for k, v in val_logs.items()}

            for k, v in val_logs.items():
                self.val_writer.add_scalar(k, v, global_step=self.global_step)
            self.val_writer.flush()

            val_acc = np.mean(val_accs)
            val_loss = np.mean(val_losses)
            print('Epoch #{} Loss: {} Accuracy: {}'.format(epoch, val_loss, val_acc))

            model.post_val_stage(val_loss)
            self.save_checkpoint(os.path.join(self.output_folder, 'last_checkpoint.pth'), model.state_dict(), optimizer.state_dict(), val_loss, epoch)
            if val_loss < self.best_loss:
                self.save_checkpoint(os.path.join(self.output_folder, 'best_checkpoint.pth'), model.state_dict(), optimizer.state_dict(), val_loss, epoch)
                self.best_loss = val_loss

    def cache_states(self):
        return {'model_state': deepcopy(self.model.state_dict()), 'optimizer_state': deepcopy(self.optimizer.state_dict())}

In [ ]:
batch_size=16
hidden_dim = 64
num_layers = 4

model = bilstm_model(hidden_dim=hidden_dim, num_layers=num_layers)
opt = torch.optim.Adam(model.parameters(), lr=1e-2)

train_loader = DataLoader(train_ds, shuffle=True, pin_memory=True, batch_size=batch_size, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, shuffle=False, pin_memory=True, batch_size=batch_size, collate_fn=collate_fn)

training = model_training(model, opt, train_loader, val_loader, batch_size=16, output_folder = '/content/drive/MyDrive/ner-2/testing33')

In [ ]:
training.train(100)

Epoch 1:   0%|          | 0/444 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

Epoch #1 Loss: 0.3154121949052324 Accuracy: 0.2663610592692207


Epoch 2:   0%|          | 0/444 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

...

Epoch #37 Loss: 0.2041931821375477 Accuracy: 0.3531748246176558


Epoch 38:   0%|          | 0/444 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

Epoch #38 Loss: 0.20679166320027137 Accuracy: 0.3600633036566309


Epoch 39:   0%|          | 0/444 [00:00<?, ?it/s]

По итогу, к 40 эпохе результаты удручающие: Loss: 0.20679166320027137 Accuracy: 0.3600633036566309

Как и к 70 эпохе: Loss: 0.2110966483549196 Accuracy: 0.43911411900874486

## Часть 4. [2 балла] Одновременное извлечение именованных сущностей и событий
1. Обучите модель для совместного извлечения именованных сущностей и триггеров событий. У модели должен быть общий энкодер (например, CNN + BiLSMT, ELMo + BiLSTM, BERT + BiLSTM) и два декодера: один отвечает за извлечение именнованных сущностей, другой отвечает за извлечение триггеров событий.

[бонус] Добавьте в модель механизм внимания, так, как это покажется вам разумным.

[бонус] Визуализируйте карты механизма внимания.

In [ ]:
# Так как работа не может быть выполнена за один раз, то снова находим наш рабочий датасет
!git clone https://github.com/dbamman/litbank.git

Cloning into 'litbank'...
remote: Enumerating objects: 1187, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1187 (delta 26), reused 22 (delta 22), pack-reused 1149
Receiving objects: 100% (1187/1187), 40.65 MiB | 24.33 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [ ]:
root = 'litbank/entities/tsv'
files = sorted([
    f"{root}/{file}"
    for file in list(os.walk(root))[0][-1]])

assert len(files) == 100

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
from string import punctuation
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from torch.utils.data import Dataset
from transformers import DistilBertTokenizer
import torch

In [ ]:
def get_sentences(path):
    df_entities = pd.read_csv(path, sep='\t', quoting=3, header=None)
    df_events = pd.read_csv(path.replace('/entities/', '/events/'), sep='\t', quoting=3, header=None)
    df = pd.concat([
            df_entities[[0, 1]],
            df_events,
        ], ignore_index=True, axis=1).dropna(subset=[0])

    words = list(df[0])
    entities_labels = list(df[1])
    events_labels = list(df[3])

    assert len(words) == len(entities_labels) and len(words) == len(events_labels), 'Что-то не так'
    sentences = [[]]
    entities_labels_sent = [[]]
    events_labels_sent = [[]]
    for i in range(len(words)):
        if words[i] not in ['.', '!', '?', '...', '']:
            if words[i] in punctuation:
                continue
            sentences[-1].append(words[i].lower())
            entities_labels_sent[-1].append(entities_labels[i])
            events_labels_sent[-1].append(events_labels[i])
        elif sentences[-1] != []:
            sentences.append([])
            entities_labels_sent.append([])
            events_labels_sent.append([])
    return sentences, entities_labels_sent, events_labels_sent

In [ ]:
def prepare_sents(files, verbose=True):
    sents, ent_labels, event_labels = [], [], []
    for f in tqdm(files, disable=not verbose):
        sent, ent, event = get_sentences(f)
        if sent[-1] == []:
            sent = sent[:-1]
            ent = ent[:-1]
            event = event[:-1]
        sents += sent
        ent_labels += ent
        event_labels += event
    return sents, ent_labels, event_labels

In [ ]:
sents, ent_labels, event_labels = prepare_sents(files)

100%|██████████| 100/100 [00:01<00:00, 87.75it/s]


In [ ]:
def split_dataset(sents, ent_labels, event_labels, train_size=0.8, random_state=42):

    data_size = len(sents)
    train_indices, remaining_indices = train_test_split(
        range(data_size), train_size=train_size, random_state=random_state
    )
    dev_indices, test_indices = train_test_split(
        remaining_indices, train_size=0.5, random_state=random_state
    )

    # Split data into train, dev, and test splits
    train_data = [sents[i] for i in train_indices], [ent_labels[i] for i in train_indices], [event_labels[i] for i in train_indices]
    dev_data = [sents[i] for i in dev_indices], [ent_labels[i] for i in dev_indices], [event_labels[i] for i in dev_indices]
    test_data = [sents[i] for i in test_indices], [ent_labels[i] for i in test_indices], [event_labels[i] for i in test_indices]

    return train_data, dev_data, test_data

In [ ]:
train_data, dev_data, test_data = split_dataset(sents, ent_labels, event_labels, 0.8, random_state=42)
train_data[0][0], train_data[1][0], train_data[2][0]

(['lead', 'him', 'not', 'into', 'temptation'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'])

In [ ]:
def flatten(lst):
    result = []
    for i in lst:
        if isinstance(i, list):
            result.extend(flatten(i))
        else:
            result.append(i)
    return result

sorted(set(flatten(ent_labels))), sorted(set(flatten(event_labels)))

(['B-FAC',
  'B-GPE',
  'B-LOC',
  'B-ORG',
  'B-PER',
  'B-VEH',
  'I-FAC',
  'I-GPE',
  'I-LOC',
  'I-ORG',
  'I-PER',
  'I-VEH',
  'O'],
 ['EVENT', 'O'])

In [ ]:
label_to_entity = {
    'B-FAC': 12,
    'B-GPE': 11,
    'B-LOC': 10,
    'B-ORG': 9,
    'B-PER': 8,
    'B-VEH': 7,
    'I-FAC': 6,
    'I-GPE': 5,
    'I-LOC': 4,
    'I-ORG': 3,
    'I-PER': 2,
    'I-VEH': 1,
    'O': 0,
}

label_to_trigger = {
    'EVENT': 1,
    'O': 0,
}

In [ ]:
def encode_sentence(tokenizer, sentence):
    encoding = tokenizer.encode_plus(
        sentence,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].squeeze()
    attention_mask = encoding['attention_mask'].squeeze()
    return input_ids, attention_mask

In [ ]:
def prepare_dataset(sentences, entities, triggers, tokenizer, label_to_entity, label_to_trigger):
    dataset = []
    for idx in range(len(sentences)):
        sentence = sentences[idx]
        entities_list = entities[idx]
        triggers_list = triggers[idx]

        input_ids, attention_mask = encode_sentence(tokenizer, sentence)
        entity_ids = torch.tensor([label_to_entity[key] for key in entities_list])
        trigger_ids = torch.tensor([label_to_trigger[key] for key in triggers_list])

        data_point = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'entity_ids': entity_ids,
            'trigger_ids': trigger_ids
        }
        dataset.append(data_point)

    return dataset

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_dataset = prepare_dataset(train_data[0], train_data[1], train_data[2], tokenizer, label_to_entity, label_to_trigger)
dev_dataset = prepare_dataset(dev_data[0], dev_data[1], dev_data[2], tokenizer, label_to_entity, label_to_trigger)
test_dataset = prepare_dataset(test_data[0], test_data[1], test_data[2], tokenizer, label_to_entity, label_to_trigger)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

Using device: cpu


In [ ]:
from torch.nn import LSTM, Linear, Module, BCEWithLogitsLoss
from transformers import DistilBertModel
from torch.nn.functional import sigmoid, one_hot
from tqdm import tqdm, trange
from IPython.display import clear_output
import gc
gc.enable()

In [ ]:
def init_weights(entity_classifier, trigger_classifier):
    initrange = 0.1
    entity_classifier.bias.data.zero_()
    entity_classifier.weight.data.uniform_(-initrange, initrange)
    trigger_classifier.bias.data.zero_()
    trigger_classifier.weight.data.uniform_(-initrange, initrange)

In [ ]:
def train_model(model, train_loader, valid_loader, num_epochs, lr, device):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = BCEWithLogitsLoss()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}")
        model.train()
        total_loss = 0

        for data in tqdm(train_loader):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            entity_ids = data['entity_ids'].to(device)
            trigger_ids = data['trigger_ids'].to(device)
            outputs = model(input_ids, attention_mask, entity_ids, trigger_ids)

            entity_loss = loss_function(outputs[0], one_hot(entity_ids, 13).float())
            trigger_loss = loss_function(outputs[1], one_hot(trigger_ids, 2).float())
            loss = entity_loss + trigger_loss
            total_loss += loss.item()

            optimizer.step()
            optimizer.zero_grad()

        avg_train_loss = total_loss / len(train_loader)
        valid_loss = evaluate_model(model, valid_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.6f} | Valid Loss: {valid_loss:.6f}")
        print()

In [ ]:
def evaluate_model(model, valid_loader, device):
    model.eval()
    total_loss = 0
    loss_function = BCEWithLogitsLoss()

    with torch.no_grad():
        for data in valid_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            entity_ids = data['entity_ids'].to(device)
            trigger_ids = data['trigger_ids'].to(device)
            outputs = model(input_ids, attention_mask, entity_ids, trigger_ids)

            entity_loss = loss_function(outputs[0], one_hot(entity_ids, 13).float())
            trigger_loss = loss_function(outputs[1], one_hot(trigger_ids, 2).float())
            loss = entity_loss + trigger_loss
            total_loss += loss.item()

    avg_valid_loss = total_loss / len(valid_loader)
    return avg_valid_loss


In [ ]:
def model_entity_event(hidden_dim, dropout_prob, device):
    bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
    lstm = LSTM(768, hidden_dim, num_layers=1, batch_first=True, dropout=dropout_prob, bidirectional=True)
    entity_classifier = Linear(hidden_dim * 2, 13)
    trigger_classifier = Linear(hidden_dim * 2, 2)
    init_weights(entity_classifier, trigger_classifier)

    def forward(input_ids, attention_mask, entity_ids, trigger_ids):
        bert_outputs = bert(input_ids, attention_mask=attention_mask)
        outputs, _ = lstm(bert_outputs.last_hidden_state)
        entity_outputs = sigmoid(entity_classifier(outputs)).squeeze()
        trigger_outputs = sigmoid(trigger_classifier(outputs)).squeeze()

        entity_outputs = entity_outputs[attention_mask == 1][1:-1]
        trigger_outputs = trigger_outputs[attention_mask == 1][1:-1]
        return entity_outputs, trigger_outputs

    return forward


In [ ]:
hidden_dim = 256
dropout_prob = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model_entity_event(hidden_dim, dropout_prob, device)
train_model(model, train_dataset, dev_dataset, num_epochs=5, lr=0.01, device=device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Epoch 1


 96%|█████████▌| 6645/6939 [2:29:59<05:36,  1.14s/it]

Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

Резюме: работа сделана и это хорошо. Это ДЗ оказалось очень тяжелым для моего уровня подготовки, но позволило узнать очень много нового.

Результаты моделей (особенно из второй части) сравнивать сложно, так как сил хватило всего на одну модель, но она показала неплохие результаты.

В выполнении работы помогли светами некотрые ребята с группы, не хватало личного опыта, еще пары людей, ну или пары недель на выполнение. Но, конечно, самая большая проблема в пробелах в знаниях.